<a href="https://colab.research.google.com/github/WeiningKan/Applied-Spatial-Statistics-Course-Project-Code/blob/main/A5_WeiningKan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5
**Due date/time:** Apr 9th Tue 23:59pm

**Instruction**: Make a copy of this notebook and read through the questions and instructions. Write down code/text to perform the analysis.

**Submission**: After you finish working on this notebook, click File->Download->Download .ipynb

Please submit a notebook named A5_your_name.ipynb on Canvas.

## Detailed instructions:

In this assignment, you will practice building Multi-level Models (MLM).

We will be re-using our data and model specifications from Assignment 3 and 4. For the same set of dependent variable (y) and independent variables (X) you will build 3 models.


**Model 1 (30%):** Fit an MLM with varying intercept only (no Xs) as follows

`model1 = Lmer('y ~ 1 + (1|state)', data=your_data)`

Describe your model output and calculate the Variance Partition Coefficient (VPC), explain your results.

**Model 2 (30%):** Fit an MLM with varying intercept and your independent variables as fixed effects as follows

`model2 = Lmer('y ~ 1 + x1 + x2 + ...xj + (1|state)', data=your_data)`

Map your state-level random effect for the intercept, and explain your MLM model output.

**Model 3 (30%):** Fit an MLM with varying intercept and both fixed and random effect for your independent variables (a varying intercept + slope model) as follows:

`model3 = Lmer('y ~ 1 + x1 + x2 + ...xj + (1 + x1 + x2 + ...xj|state)', data=your_data)`

Map all of your state-level random effects, and explain your MLM model output.

**Final comparison (10%):** compare AIC, conditional and marginal R2 in three models.

Note: replace x1, x2... in the model with the column name of your independent variable.


In [ ]:
pip install -q pymer4==0.7.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 769.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q libpysal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.4 MB/s eta 0:00:00


In [ ]:
pip install -q esda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install pandas==1.3.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 20.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import rpy2.ipython
%load_ext rpy2.ipython

import logging
# Suppress Python warnings for rpy2 callbacks
logging.getLogger('rpy2.rinterface_lib.callbacks').setLevel(logging.ERROR)

In [ ]:
%%time
#This may take a while (~5 mins)
%R install.packages("lme4")
%R install.packages("lmerTest");

CPU times: user 3.41 s, sys: 376 ms, total: 3.79 s
Wall time: 5min 14s


In [ ]:
import warnings

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import libpysal as ps
from libpysal.weights import Queen
from esda.moran import Moran

from pymer4.models import Lmer

Code to read in data and merge them into one final `GeoDataFrame`.

In [ ]:
health_url = "https://raw.github.com/Ziqi-Li/GIS5122/main/data/cdc_health_county.csv"
health = pd.read_csv(health_url)
health

,LocationID,ACCESS2,ARTHRITIS,BINGE,CANCER,CASTHMA,CERVICAL,CHD,CHECKUP,COLON_SCREEN,...,DIABETES,KIDNEY,LPA,MAMMOUSE,MHLTH,OBESITY,PHLTH,SLEEP,STROKE,TEETHLOST
0,1001,14.3,29.3,14.1,6.5,10.2,86.4,6.9,77.0,69.5,...,11.3,2.8,26.4,77.5,15.8,35.5,13.3,38.4,3.4,16.3
1,1003,13.6,27.3,17.3,6.7,9.6,87.3,6.0,75.7,67.5,...,9.5,2.6,23.5,74.0,14.8,29.4,11.6,36.4,3.0,13.4
2,1005,21.8,33.2,12.3,6.1,11.7,84.1,9.0,78.5,62.1,...,16.9,3.9,36.8,75.9,19.0,40.8,18.1,40.9,5.1,27.1
3,1007,17.3,30.6,15.0,6.4,10.3,84.8,7.6,76.3,65.7,...,12.6,3.0,31.2,73.3,17.1,38.4,15.1,40.1,3.8,20.7
4,1009,17.0,30.1,15.1,6.6,10.2,84.1,7.5,74.2,63.9,...,11.3,2.9,30.1,71.5,17.1,33.7,14.8,38.8,3.5,19.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,17.7,24.6,18.6,6.5,9.0,83.7,5.7,66.4,56.3,...,8.5,2.5,23.6,63.7,12.8,33.2,11.5,35.4,2.9,15.7
3138,56039,13.1,18.9,20.3,6.4,8.0,86.9,4.8,67.2,63.4,...,6.6,2.2,16.2,70.5,10.2,18.8,8.9,27.1,2.3,9.9
3139,56041,17.9,22.9,17.4,6.4,9.6,81.8,6.3,65.2,54.4,...,9.1,2.7,24.3,61.1,14.2,33.6,12.5,35.9,3.1,19.1
3140,56043,18.0,22.6,18.3,6.4,9.3,83.3,5.9,65.8,53.5,...,8.6,2.6,22.7,63.6,13.3,28.3,11.7,32.4,3.0,16.3


In [ ]:
socio_demo_url = "https://raw.github.com/Ziqi-Li/GIS5122/main/data/acs_2019.csv"
socio_demo = pd.read_csv(socio_demo_url)

In [ ]:
counties_url = "https://raw.github.com/Ziqi-Li/gis5122/master/data/us_counties.geojson"
counties = gpd.read_file(counties_url)

In [ ]:
socio_demo_health = pd.merge(socio_demo, health, left_on = "county_id", right_on = "LocationID")

In [ ]:
print(gdf_socio_demo_health.columns)


Index(['GEOID', 'STATEFP', 'NAME_x', 'county_id', 'geometry', 'state',
       'county', 'NAME_y', 'proj_X', 'proj_Y', 'total_pop', 'sex_ratio',
       'pct_black', 'pct_hisp', 'pct_bach', 'median_income', 'pct_65_over',
       'pct_age_18_29', 'gini', 'pct_manuf', 'ln_pop_den', 'pct_fb',
       'pct_uninsured', 'LocationID', 'ACCESS2', 'ARTHRITIS', 'BINGE',
       'CANCER', 'CASTHMA', 'CERVICAL', 'CHD', 'CHECKUP', 'COLON_SCREEN',
       'COPD', 'COREM', 'COREW', 'CSMOKING', 'DENTAL', 'DIABETES', 'KIDNEY',
       'LPA', 'MAMMOUSE', 'MHLTH', 'OBESITY', 'PHLTH', 'SLEEP', 'STROKE',
       'TEETHLOST'],
      dtype='object')


In [ ]:
gdf_socio_demo_health = pd.merge(counties, socio_demo_health, on = "county_id")
gdf_socio_demo_health[['median_income']] = gdf_socio_demo_health[['median_income']]/10000

In [ ]:
state = gdf_socio_demo_health.dissolve(by='STATEFP')

In [ ]:
gdf_socio_demo_health

Your code will start here, and your analysis will be based on this GeoDataFrame `gdf_income_health`.

# **Model 1 (30%): Fit an MLM with varying intercept only (no Xs) as follows**

model1 = Lmer('y ~ 1 + (1|state)', data=your_data)

Describe your model output and calculate the Variance Partition Coefficient (VPC), explain your results.

In [ ]:
gdf_socio_demo_health_df = gdf_socio_demo_health.drop(columns = 'geometry')

In [ ]:
state = gdf_socio_demo_health.dissolve(by='STATEFP').geometry.boundary

In [ ]:
model_1 = Lmer('OBESITY ~ 1 + (1|state)', data = gdf_socio_demo_health_df)

In [ ]:
model_1.fit()

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [ ]:
def r_square(model):
    var_a = np.sum(model.ranef_var.Var[:-1]) #Sum of variances from random effects
    var_e = model_1.ranef_var.Var[-1] #Error variance
    var_f = np.sum(model.design_matrix*model.coefs.Estimate,axis=1).var() #Sum of variances from fixed effects

    marginal = var_f/(var_f+var_a+var_e)

    conditional = (var_f+var_a)/(var_f+var_a+var_e)

    print("marginal R2:",marginal)
    print("conditional R2:",conditional)
    return


In [ ]:
r_square(model_1)

*   There are 3103 observations
*   There are 48 groups
*   The model has an AIC of 16389.629
*   The state-level random effect has an estimated variance of 12.854 with a Std of 3.585
*   The residual has an estimated variance 10.828 with a Std of 3.291
*   The VPC = 12.854/(12.854+10.828) = 54.3%. This means that 54.3% of the variance in the health can be attributed to the fact that counties belong to different states.
*   There are no predictors, only one intercept here, and the intercept estimate is 33.878, and it is statistically significance (p-value = 0.0 < 0.05)







# **Model 2 (30%): Fit an MLM with varying intercept and your independent variables as fixed effects as follows**

model2 = Lmer('y ~ 1 + x1 + x2 + ...xj + (1|state)', data=your_data)

Map your state-level random effect for the intercept, and explain your MLM model output.

In [ ]:
model_2 = Lmer('OBESITY ~ 1 + sex_ratio + median_income + gini + pct_age_18_29 + pct_65_over + pct_bach + (1|state)', data=gdf_socio_demo_health_df)
model_2.fit()

In [ ]:
def r_square(model):
    var_a = np.sum(model.ranef_var.Var[:-1]) #Sum of variances from random effects
    var_e = model_2.ranef_var.Var[-1] #Error variance
    var_f = np.sum(model.design_matrix*model.coefs.Estimate,axis=1).var() #Sum of variances from fixed effects

    marginal = var_f/(var_f+var_a+var_e)

    conditional = (var_f+var_a)/(var_f+var_a+var_e)

    print("marginal R2:",marginal)
    print("conditional R2:",conditional)
    return


In [ ]:
r_square(model_2)

*   There are 3103 observations
*   There are 48 groups
*   The model investigates the relationship between obesity rates (OBESITY) and several predictors (sex_ratio, median_income, gini, pct_age_18_29, pct_65_over, pct_bach)
*   The model has an AIC of 14044.006
*   The state-level random effect has an estimated variance of 5.430 with a Std of 2.330
*   The residual has an estimated variance 5.046 with a Std of 2.246
*   The VPC = 5.430/(5.430+5.046) = 51.8%. This means that 51.8% of the variance in the health can be attributed to the fact that counties belong to different states.
*   Sex_ratio, pct_age_18_29 do not have a statistically significant effect on obesity rates (P-val > 0.05). median_income, gini, pct_65_over, pct_bach show statistically significant effects. For instance, median_income has a negative relationship with obesity rates, meaning higher income is associated with lower obesity rates.

# **Model 3 (30%): Fit an MLM with varying intercept and both fixed and random effect for your independent variables (a varying intercept + slope model) as follows:**

model3 = Lmer('y ~ 1 + x1 + x2 + ...xj + (1 + x1 + x2 + ...xj|state)', data=your_data)

Map all of your state-level random effects, and explain your MLM model output.

In [ ]:
warnings.filterwarnings('ignore')

model_3 = Lmer('OBESITY ~ 1 + sex_ratio + median_income + gini + pct_age_18_29 + pct_65_over + pct_bach + (1 + sex_ratio + median_income + gini + pct_age_18_29 + pct_65_over + pct_bach|state)', data=gdf_socio_demo_health_df)

model_3.fit()

In [ ]:
def r_square(model):
    var_a = np.sum(model.ranef_var.Var[:-1]) #Sum of variances from random effects
    var_e = model_3.ranef_var.Var[-1] #Error variance
    var_f = np.sum(model.design_matrix*model.coefs.Estimate,axis=1).var() #Sum of variances from fixed effects

    marginal = var_f/(var_f+var_a+var_e)

    conditional = (var_f+var_a)/(var_f+var_a+var_e)

    print("marginal R2:",marginal)
    print("conditional R2:",conditional)
    return


In [ ]:
r_square(model_3)

The variance of 14.915 for Intercept. This value indicates there's considerable variability in the baseline obesity rates across states, suggesting that states significantly differ in their starting points regarding obesity rates. The correlations between random effects, such as between the intercept and median_income (-0.827), suggest how changes in one variable might be associated with changes in another across states. A negative correlation implies that as the baseline obesity rate increases, the effect of median_income on reducing obesity rates also increases, but this relationship is not uniform across all states.

In [ ]:
model_3.ranef.hist()

In [ ]:
variables = ['sex_ratio', 'median_income', 'gini', 'pct_age_18_29', 'pct_65_over', 'pct_bach']
for i, var in enumerate(variables, start=1):
    plt.figure()
    plt.scatter(model_3.ranef.iloc[:, 0], model_3.ranef.iloc[:, i])
    plt.xlabel("Random effect of intercept")
    plt.ylabel(f"Random effect of {var}")
    plt.title(f"Random Effects: Intercept vs. {var}")
    plt.show()


*  median_income, pct_age_18_29, pct_65_over： A negative correlation indicates that when the intercept is larger, the slope is (smaller) flatter across the states.

*  gini, pct_bach： A negative correlation indicates that when the intercept is larger, the slope is (bigger) flatter across the states.

In [ ]:
state.plot(model_3.ranef.iloc[:,0].values,legend=True)
plt.title("Varying intercept")

In [ ]:
state.plot(model_3.ranef.iloc[:,1].values,legend=True)
plt.title("Varying slope of sex_ratio")

In [ ]:
state.plot(model_3.ranef.iloc[:,2].values,legend=True)
plt.title("Varying slope of median_income")

In [ ]:
state.plot(model_3.ranef.iloc[:,3].values,legend=True)
plt.title("Varying slope of gini")

In [ ]:
state.plot(model_3.ranef.iloc[:,4].values,legend=True)
plt.title("Varying slope of pct_age_18_29")

In [ ]:
state.plot(model_3.ranef.iloc[:,5].values,legend=True)
plt.title("Varying slope of pct_65_over")

In [ ]:
state.plot(model_3.ranef.iloc[:,6].values,legend=True)
plt.title("Varying slope of pct_bach")

# **Final comparison (10%): compare AIC, conditional and marginal R2 in three models.**

Note: replace x1, x2... in the model with the column name of your independent variable.

In [ ]:
data = {
    'Model': ['Model_1', 'Model_2', 'Model_3'],
    'AIC': [16389.629, 14044.006, 13896.925],
    'Marginal R2': [0.000, 0.439, 0.158],
    'Conditional R2': [0.542, 0.730, 0.912]
}

df = pd.DataFrame(data)

df.set_index('Model')

df

**Model 1**  

* The model only considers random intercepts for each state, reflecting variations in obesity rates across states without considering any fixed predictors.
* It has the highest AIC value 16389.629, suggesting it's the least fit among the three. Despite the lack of fixed effect predictors, the conditional R-squared suggests that state-level differences account for over half of the variability in obesity rates.

**Model 2**  
* This model adds fixed effects for various demographic and socioeconomic predictors. It significantly improves upon Model_1. This model has a much lower AIC, suggesting a better fit than Model_1.
* The marginal R2 value is 0.439, meaning that nearly 44% of the variance in the outcome is explained by the fixed effects alone. The conditional R2 is higher at 0.730, indicating a substantial amount of variance explained by the model when including random effects.

**Model 3**  
* The most complex model includes both fixed effects and random slopes for each predictor within states, allowing for a nuanced understanding of how these relationships vary across states.
* It has the lowest AIC, which indicates the **best fit** of the three models according to this criterion.
* Its marginal R2 is lower than that of Model_2, which might indicate that the inclusion of random slopes for each predictor within states doesn't add much to the explanation provided by the fixed effects alone. The conditional R2 is the highest at 0.912, which means that the model, with both fixed and random effects, accounts for most of the variance in the outcome.
